In [2]:
import json, os, itertools
from typing import List, Dict, Any, Iterable, Union
from datasets import load_dataset

PARAPHRASED_PATH = "paraphrased_25k.jsonl"

# Load paraphrased dataset (JSONL or JSON array) into `paraphrased`
import json, os

PARAPHRASED_PATH = "paraphrased_pruned_owl-final.jsonl"  # <-- set your path here

def read_json_maybe_jsonl(path: str):
    if path.lower().endswith(".jsonl"):
        rows = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                try:
                    rows.append(json.loads(line))
                except Exception:
                    pass
        return rows
    else:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            return data
        # if it's a dict, try to grab the longest list-like value
        if isinstance(data, dict):
            best = None
            for v in data.values():
                if isinstance(v, list) and (best is None or len(v) > len(best)):
                    best = v
            if best is not None:
                return best
        raise ValueError("Unsupported JSON structure for paraphrased dataset.")

paraphrased = read_json_maybe_jsonl(PARAPHRASED_PATH)
print(f"Loaded paraphrased items: {len(paraphrased):,}")
print("Sample keys:", list(paraphrased[0].keys()) if paraphrased else "(empty)")

from datasets import load_dataset
alpaca = load_dataset("tatsu-lab/alpaca", split="train")

PREFIX = "Additional context: "
ONLY_IF_NONEMPTY = True

def new_instruction(old_instr, alp_input):
    if ONLY_IF_NONEMPTY and not (alp_input and alp_input.strip()):
        return old_instr
    return f"{old_instr.rstrip()}\n\n{PREFIX}{alp_input.strip()}" if alp_input else old_instr

updated = []
for ex in paraphrased:  # every row
    ri = ex.get("row_index")
    if isinstance(ri, int) and 0 <= ri < len(alpaca):
        alp_inp = alpaca[ri].get("input") or ""
        ex = dict(ex)  # copy
        ex["instruction"] = new_instruction(ex.get("instruction", ""), alp_inp)
    updated.append(ex)

# save
with open("paraphrased_25k.updated.jsonl", "w", encoding="utf-8") as f:
    for r in updated:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

Loaded paraphrased items: 21,426
Sample keys: ['row_index', 'instruction', 'original_output', 'paraphrased_output', 'model']


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [2]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.1/564.1 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 47.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.
